In [5]:
import pickle
import pybedtools
import kipoiseq
import os
import pandas as  pd
import numpy as np
from data_utils import *

# Collecting CAGE counts at TSS:

**For each gene collect CAGE counts at all alternative TSS**

1. get bin information for all alternative TSS
2. get input region that overlaps with each TSS
3. extract the count value at the TSS bin from targets and predictions for the overlapping input region
4. sum all raw counts at all alternative TSS bin (+ 2 neighbouring bins)

We are left with one single gene expression value per gene. 

## 1. Find overlap between unique TSS and test set regions

In [6]:
odcf = False
if odcf:
    data_dir ="/omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj"
else:
    data_dir = "/data/mikulik/mnt/gcs_basenj/"


In [7]:
# read in all unique tss 
tss = pd.read_csv(os.path.join(data_dir, "unique_tss.csv"), header=0, index_col=0)
print(tss.shape)
tss.head()



(213986, 16)


/tmp/ipykernel_124460/3491214571.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tss = pd.read_csv(os.path.join(data_dir, "unique_tss.csv"), header=0, index_col=0)


,transcript_stable_id,version,transcript_type,chrom,strand,start,end,tss,ensembl_canonical,RefSeq_match_transcript_MANE_select,TSL,gene_stable_id,gene_name,strand_aware_start,strand_aware_end,tss_coord
10474,ENST00000431340,1,unprocessed_pseudogene,Y,1,8280198,8282209,8280198,1.0,NaN,tslNA,ENSG00000215601,TSPY24P,8280198,8282209,chrY:8280198_8280199
10478,ENST00000415010,1,processed_pseudogene,Y,-1,7853686,7854876,7854876,1.0,NaN,tslNA,ENSG00000215603,ZNF92P1Y,7854876,7853686,chrY:7854876_7854877
10480,ENST00000430062,1,processed_pseudogene,Y,-1,5207215,5208069,5208069,1.0,NaN,tslNA,ENSG00000231341,VDAC1P6,5208069,5207215,chrY:5208069_5208070
10484,ENST00000449381,1,unprocessed_pseudogene,Y,-1,9610571,9621276,9621276,1.0,NaN,tslNA,ENSG00000231436,RBMY3AP,9621276,9610571,chrY:9621276_9621277
10489,ENST00000651261,1,processed_transcript,Y,-1,7694707,7809267,7809267,NaN,NaN,NaN,ENSG00000234795,RFTN1P1,7809267,7694707,chrY:7809267_7809268


Reduce the length of the test set regions to 114,688 bp and write them to a bed file. The same for the unique tss from Karollus et al. 

In [8]:
def write_input_regions_to_bed(data_dir, species, subset):
    '''
    data_dir (str): path to input region bed file for both species
    seq_bed (str): name of the bed file that stores the input regions for the species specified
    species (str): one of human or mouse
    subset (str): on of train/test/valid
    '''
    # read in the input regions
    region_df = pd.read_csv(os.path.join(data_dir, species, "sequences.bed"), sep="\t", header=None)
    region_df.columns = ["Chromosome", "Start", "End", "subset"]
    region_df.reset_index(inplace=True)
    region_df.columns = ["index", "Chromosome", "Start", "End", "subset"]
    region_df = region_df[["Chromosome", "Start", "End", "subset", "index"]]
    region_df_test = region_df[region_df["subset"] == subset].copy()
    print(region_df_test.shape)

    # crop the sequence length to 114688, to contain only the center 896 bins for which we make predictions
    seq_len_crop = 131072 - (128 * 64 * 2)
    print(f"Crop to {seq_len_crop}")
    for row, region in region_df_test.iterrows():
        interval = kipoiseq.Interval(region.Chromosome, region.Start, region.End).resize(seq_len_crop) 
        region_df_test.at[row, "Start"] = interval.start
        region_df_test.at[row, "End"] = interval.end
        assert region_df_test.at[row, "End"] - region_df_test.at[row, "Start"] == seq_len_crop
    region_bed = pybedtools.BedTool.from_dataframe(region_df_test)
    print(f'Save at {os.path.join(data_dir, species, f"{subset}_{species}_center_114688.bed")}')
    region_bed.saveas(os.path.join(data_dir, species, f"{subset}_{species}_center_114688.bed"))

In [9]:
# write all region subsets to separate bed files 
write_input_regions_to_bed(data_dir, "human", "train")
write_input_regions_to_bed(data_dir, "human", "test")
write_input_regions_to_bed(data_dir, "human", "valid")


(34021, 5)
Crop to 114688
Save at /data/mikulik/mnt/gcs_basenj/human/train_human_center_114688.bed
(1937, 5)
Crop to 114688
Save at /data/mikulik/mnt/gcs_basenj/human/test_human_center_114688.bed
(2213, 5)
Crop to 114688
Save at /data/mikulik/mnt/gcs_basenj/human/valid_human_center_114688.bed


In [10]:
train_regions = pd.read_csv(os.path.join(data_dir, "human", "train_human_center_114688.bed"),sep="\t",header=None)
train_regions

,0,1,2,3,4
0,chr18,936578,1051266,train,0
1,chr4,113639139,113753827,train,1
2,chr11,18435912,18550600,train,2
3,chr16,85813873,85928561,train,3
4,chr3,158394380,158509068,train,4
...,...,...,...,...,...
34016,chr7,50523314,50638002,train,34016
34017,chr7,135610961,135725649,train,34017
34018,chr4,189012390,189127078,train,34018
34019,chr4,10446291,10560979,train,34019


Reduce the length of the test set regions to 114,688 bp and write them to a bed file. The same for the unique tss from Karollus et al. 

In [11]:
# write the tss coordinates from Karollus to a bed file
tss_sub = tss[["tss_coord", "gene_stable_id"]].copy()
tss_sub[["Chromosome", "tmp"]] = tss_sub.tss_coord.str.split(":", n=1, expand=True)
tss_sub[["Start", "End"]] = tss_sub.tmp.str.split("_", expand=True)
tss_bed = pybedtools.BedTool.from_dataframe(tss_sub[["Chromosome", "Start", "End", "gene_stable_id"]])
print(type(tss_bed))
print(len(tss_bed))
tss_bed
# save
save = False
if save:
    tss_bed.saveas(os.path.join(data_dir, "tss_all.bed"))

<class 'pybedtools.bedtool.BedTool'>
213986


In [12]:
# write only the tss coordinates of protein-coding genes from Karollus to a bed file
tss = tss[tss.transcript_type == "protein_coding" ]
tss_sub = tss[["tss_coord", "gene_stable_id"]].copy()
tss_sub[["Chromosome", "tmp"]] = tss_sub.tss_coord.str.split(":", n=1, expand=True)
tss_sub[["Start", "End"]] = tss_sub.tmp.str.split("_", expand=True)
tss_bed = pybedtools.BedTool.from_dataframe(tss_sub[["Chromosome", "Start", "End", "gene_stable_id"]])
print(type(tss_bed))
print(len(tss_bed))
tss_bed
# save
save = False
if save:
    tss_bed.saveas(os.path.join(data_dir, "tss_protein_coding.bed"))

<class 'pybedtools.bedtool.BedTool'>
74633


#### Find TSSs in the input sequences

Now  we want to know which of the regions in the test set overlap with at least one of the TSSs.

The bedtools overlap can be runf with the script `/omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/bedtools_overlap_karollus_tss.py`. Make sure to use the tss coordinates first and the input region bed file second! Otherwise the `get_overlap` function will not work anymore. Some of the regions in the test set do not contain any of the TSSs!


**For all tss**

```
python bedtools_overlap_karollus_tss.py tss_all.bed /human/train_human_center_114688.bed /human/overlap_train.bed /omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/
```

```
python bedtools_overlap_karollus_tss.py tss_all.bed /human/valid_human_center_114688.bed /human/overlap_valid.bed /omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/
```


**For only protein-coding gene tss**
```
python bedtools_overlap_karollus_tss.py tss_protein_coding.bed /human/train_human_center_114688.bed /human/overlap_train_protein_coding.bed /omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/
```
```
python bedtools_overlap_karollus_tss.py tss_protein_coding.bed /human/valid_human_center_114688.bed /human/overlap_valid_protein_coding.bed /omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/
```
```
python bedtools_overlap_karollus_tss.py tss_protein_coding.bed /human/test_human_center_114688.bed /human/overlap_test_protein_coding.bed /omics/groups/OE0540/internal/users/mikulik/master_thesis/data/gcs_basenj/
```


In [13]:
def get_overlap(data_dir, file_name):
    overlap_tss = pd.read_csv(os.path.join(data_dir, f"{file_name}.bed"), sep="\t", header=None)
    overlap_tss.columns = ["tss_chrom", "tss_start", "tss_end", "gene_id",  "test_region_chr", "test_region_start", "test_region_end", "subset", "region_index",  "overlap_size"]
    overlap_tss["region_coord"] = overlap_tss.test_region_chr + ":" + overlap_tss.test_region_start.astype("str") + "_" + overlap_tss.test_region_end.astype("str")
    overlap_tss["tss_coord"] = overlap_tss.tss_chrom + ":" + overlap_tss.tss_start.astype("str") + "_" + overlap_tss.tss_end.astype("str")
    overlap_tss["seq_len"] = overlap_tss.test_region_end -  overlap_tss.test_region_start
    # calculate the position of the TSS within the test region
    overlap_tss["pos_tss"] = np.abs(overlap_tss.test_region_start - overlap_tss.tss_start)
    assert 0 < overlap_tss.pos_tss.max() < 114688 
    # convert the position of the TSS to the 128 bp bin number
    overlap_tss["bin"] = np.round(overlap_tss.pos_tss // 128)
    print((overlap_tss.bin < 0).sum())
    assert all(overlap_tss.bin >= 0)

    overlap_tss  = overlap_tss.sort_values(by="region_index", ascending=True)
    #print(f"The number of input sequences in the test set is {len(region_df_test)}, while the number of regions which have at least one TSS overlapping is {len(overlap_tss.region_index.unique())}")
    assert overlap_tss.bin.max() < 896
    assert overlap_tss.bin.min() >= 0
    return overlap_tss


In [14]:
regions = pd.read_csv(os.path.join(data_dir, "human", "sequences.bed"), sep="\t", header=None)
regions.columns = ["chr", "start", "end","subset"]
regions[regions.subset =="test"]

,chr,start,end,subset
36234,chr10,37555537,37686609,test
36235,chr14,87048845,87179917,test
36236,chrX,136527085,136658157,test
36237,chr11,34042349,34173421,test
36238,chr19,10352757,10483829,test
...,...,...,...,...
38166,chr19,33204702,33335774,test
38167,chr14,41861379,41992451,test
38168,chr19,30681544,30812616,test
38169,chr14,61473198,61604270,test


In [15]:
overlap_tss = get_overlap(os.path.join(data_dir, "human"), "overlap_test_protein_coding")
overlap_tss.region_index.isin(regions[regions.subset =="test"].index).all()

0


True

In [16]:
ds = EnformerDataset("human", 
                     "test",
                     131072,
                     data_dir, 
                     human_fasta_path = os.path.join(data_dir, "hg38.ml.fa"), 
                     mouse_fasta_path = os.path.join(data_dir, "mm10.ml.fa"), 
                     random=False)


In [17]:
overlap_tss.region_index.isin(ds.input_sequence.region_df["index"]).all()

True

In [18]:
def update_gene_dict(single_region_df:pd.DataFrame, gene_dict:dict, target:np.array, prediction:np.array):
    for i, row in single_region_df.iterrows():
        gene = row.gene_id
        bin = int(row.bin)
        counts_target = target[:, bin-1 : bin+2, :].sum(axis=1).squeeze()
        counts_prediction = prediction[:, bin-1 : bin+2, :].sum(axis=1).squeeze()
        #print(counts_prediction)
        gene_dict["pred"][gene] = gene_dict["pred"][gene] + counts_prediction
        gene_dict["tar"][gene] = gene_dict["tar"][gene] + counts_target
        #print(gene_dict)
